## Product Families

The CMO is interested in understanding how the sales of different product families are affected by promotional campaigns. To do so, for each product family, show the total number of units sold, as well as the percentage of units sold that had a valid promotion among total units sold. If there are NULLS in the result, replace them with zeroes. Promotion is valid if it's not empty and it's contained inside promotions table.

## `facebook_products` Table Schema

This table contains information about various products.

| Column Name      | Type     | Description                               |
| :--------------- | :------- | :---------------------------------------- |
| `product_id`     | `int64`  | Unique identifier for the product         |
| `product_class`  | `object` | Classification of the product (e.g., 'Electronics', 'Apparel') |
| `brand_name`     | `object` | Brand name of the product                 |
| `is_low_fat`     | `object` | Indicates if the product is low-fat ('Y'/'N') |
| `is_recyclable`  | `object` | Indicates if the product is recyclable ('Y'/'N') |
| `product_category`| `int64`  | Category ID of the product                |
| `product_family` | `object` | Family of the product (e.g., 'Smartphones', 'Laptops') |

---

## `facebook_sales_promotions` Table Schema

This table holds details about different sales promotions.

| Column Name      | Type          | Description                                  |
| :--------------- | :------------ | :------------------------------------------- |
| `promotion_id`   | `int64`       | Unique identifier for the promotion          |
| `start_date`     | `datetime64[ns]`| The start date and time of the promotion     |
| `end_date`       | `datetime64[ns]`| The end date and time of the promotion       |
| `media_type`     | `object`      | The type of media used for the promotion (e.g., 'TV', 'Online', 'Print') |
| `cost`           | `int64`       | The cost of the promotion                    |

---

## `facebook_sales` Table Schema

This table records individual sales transactions.

| Column Name      | Type          | Description                                  |
| :--------------- | :------------ | :------------------------------------------- |
| `product_id`     | `int64`       | Foreign key referencing `facebook_products`  |
| `promotion_id`   | `int64`       | Foreign key referencing `facebook_sales_promotions` |
| `cost_in_dollars`| `int64`       | The cost of the item to the customer in dollars |
| `customer_id`    | `int64`       | Unique identifier for the customer           |
| `date`           | `datetime64[ns]`| The date and time of the sale                |
| `units_sold`     | `int64`       | The number of units sold in this transaction |

Used Gemini/ChatGPT to create dataset

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Parameters
num_channels = 10
num_posts = 100

# Generate channels dataframe
channel_ids = np.arange(1, num_channels + 1)
channel_types = [f"Channel_{i}" for i in channel_ids]
channel_names = np.random.choice(['News', 'Entertainment', 'Sports', 'Education'], size=num_channels)

channels = pd.DataFrame({
    'channel_id': channel_ids,
    'channel_name': channel_names,
    'channel_type': channel_types
})

# Generate posts dataframe
post_ids = np.arange(1, num_posts + 1)
post_channel_ids = np.random.choice(channel_ids, size=num_posts)
created_ats = [datetime.now() - timedelta(days=random.randint(0, 365)) for _ in range(num_posts)]
likes = np.random.randint(0, 1000, size=num_posts)
shares = np.random.randint(0, 500, size=num_posts)
comments = np.random.randint(0, 300, size=num_posts)

posts = pd.DataFrame({
    'post_id': post_ids,
    'channel_id': post_channel_ids,
    'created_at': created_ats,
    'likes': likes,
    'shares': shares,
    'comments': comments
})



### Solution

In [ ]:
posts = posts[['post_id','channel_id','created_at','likes']]
channels = channels[['channel_id','channel_name']]

posts = pd.merge(posts,channels,on='channel_id',how='left')

posts.drop(columns={'channel_id'},inplace=True)

posts['rank'] = posts.groupby('channel_name')['likes'].rank(ascending=False, method='min')
posts = posts.sort_values(by=['channel_name','likes'],ascending=False)

posts = posts[posts['rank'] <= 3]

posts